# README

The ultimate aim of this notebook is to construct a **naive** *implied order book* for SPX options (options on the S&P500). We will explain what this means in more detail. The data comes from the Chicago Board Options Exchange (CBOE). The data is delayed (hence why it's free!). Here's the link to the page where the data comes from. https://www.cboe.com/delayed_quotes/spx/quote_table

When we make a call to the API we get back a table containing all option contracts on the S&P500 (one row per contract), with some info about the contract:
* The ID tells us the expiry, whether it's a put or a call and the strike price.
* The current bid and ask prices, as well as the bid and ask sizes
* Implied Vol (need to work out whether this is from bid, ask, mid or something else - may need to write my own Black-Scholes function for this)
* Open Interest (total number of open contracts)
* Volume
* The Greeks - delta, gamma, vega, theta, rho, theo
* Some metrics about how the price has changed
* Last traded time and last traded price. This can help to show liquidity.

What this dataset doesn't tell you... EXPLAIN WHY IT'S NAIVE

https://perfiliev.com/blog/how-to-calculate-gamma-exposure-and-zero-gamma-level/

# Imports

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

# Fetch Data

In [2]:
# Symbol for SPX options
symbol = '_SPX'

# Start session
s = requests.Session()

# Make API call and check status code
response = s.get(f'https://cdn.cboe.com/api/global/delayed_quotes/options/{symbol}.json')
if response.status_code != 200:
    raise Exception(f"Unable to fetch data. Status code is {response.status_code}")

# Get data as json
data = json.loads(response.content)

# Close session
s.close()

In [3]:
# Get timestamp
timestamp = datetime.strptime(data['timestamp'], "%Y-%m-%d %H:%M:%S")

# Get index price
index_price = data['data']['current_price']

# Get options data
df = pd.DataFrame(
    data=data['data']['options']
)

df['Index Price'] = index_price
df['timestamp'] = timestamp

# Extract info from the ID column
without_w = df['option'].str.replace("W", "")
df['Expiry'] = without_w.str[3:9].astype(int)
df['Type'] = without_w.str[9]
df['Strike'] = without_w.str[11:-3].astype(int)

# Calculate moneyness
df['Moneyness'] = (df['Index Price'] - df['Strike']) / df['Strike']

# Order by expiry
df.sort_values(by='Expiry')

,option,bid,bid_size,ask,ask_size,iv,open_interest,volume,delta,gamma,...,last_trade_price,last_trade_time,percent_change,prev_day_close,Index Price,timestamp,Expiry,Type,Strike,Moneyness
8032,SPXW241108C01600000,4392.70,11.0,4399.0,11.0,0.0000,16.0,0.0,1.0000,0.0000,...,4369.53,2024-11-07T12:42:34,0.0000,4377.449951,5996.3101,2024-11-08 16:41:29,241108,C,1600,2.747694
8417,SPXW241108P05880000,0.05,112.0,0.1,656.0,0.3357,4346.0,3689.0,-0.0048,0.0003,...,0.10,2024-11-08T11:23:24,-89.1892,0.925000,5996.3101,2024-11-08 16:41:29,241108,P,5880,0.019781
8416,SPXW241108C05880000,112.20,12.0,117.8,11.0,0.3617,913.0,72.0,0.9952,0.0003,...,114.93,2024-11-08T11:26:03,14.5862,100.299999,5996.3101,2024-11-08 16:41:29,241108,C,5880,0.019781
8415,SPXW241108P05875000,0.05,24.0,0.1,721.0,0.3487,3596.0,2093.0,-0.0045,0.0003,...,0.05,2024-11-08T11:26:07,-93.9394,0.825000,5996.3101,2024-11-08 16:41:29,241108,P,5875,0.020649
8414,SPXW241108C05875000,117.70,12.0,124.3,12.0,0.0000,4501.0,1012.0,0.9955,0.0003,...,121.22,2024-11-08T10:53:04,15.0641,105.349998,5996.3101,2024-11-08 16:41:29,241108,C,5875,0.020649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7936,SPX291221C03700000,2873.50,5.0,2953.5,5.0,0.2401,0.0,0.0,0.9616,0.0000,...,0.00,None,0.0000,2897.199951,5996.3101,2024-11-08 16:41:29,291221,C,3700,0.620624
7935,SPX291221P03600000,108.10,71.0,118.5,18.0,0.2449,324.0,0.0,-0.0670,0.0000,...,119.72,2024-11-06T12:28:35,0.0000,114.950001,5996.3101,2024-11-08 16:41:29,291221,P,3600,0.665642
7934,SPX291221C03600000,2945.00,5.0,3025.0,5.0,0.2428,1.0,0.0,0.9669,0.0000,...,0.00,None,0.0000,2968.399902,5996.3101,2024-11-08 16:41:29,291221,C,3600,0.665642
7948,SPX291221C04300000,2457.10,5.0,2537.1,5.0,0.2257,0.0,0.0,0.9240,0.0001,...,0.00,None,0.0000,2481.100098,5996.3101,2024-11-08 16:41:29,291221,C,4300,0.394491


In [4]:
df

,option,bid,bid_size,ask,ask_size,iv,open_interest,volume,delta,gamma,...,last_trade_price,last_trade_time,percent_change,prev_day_close,Index Price,timestamp,Expiry,Type,Strike,Moneyness
0,SPX241115C00200000,5789.8,24.0,5799.80,24.0,0.0000,16.0,0.0,1.0000,0.0000,...,5655.41,2024-10-18T12:12:30,0.000000,5774.800049,5996.3101,2024-11-08 16:41:29,241115,C,200,28.981550
1,SPX241115P00200000,0.0,0.0,0.05,452.0,6.7584,116.0,0.0,0.0000,0.0000,...,0.00,None,0.000000,0.025000,5996.3101,2024-11-08 16:41:29,241115,P,200,28.981550
2,SPX241115C00400000,5590.0,24.0,5599.90,24.0,6.7531,1.0,0.0,1.0000,0.0000,...,5428.84,2024-10-21T11:17:52,0.000000,5575.049805,5996.3101,2024-11-08 16:41:29,241115,C,400,13.990775
3,SPX241115P00400000,0.0,0.0,0.05,413.0,5.3312,972.0,0.0,0.0000,0.0000,...,0.05,2024-10-31T15:47:22,0.000000,0.025000,5996.3101,2024-11-08 16:41:29,241115,P,400,13.990775
4,SPX241115C00600000,5390.1,24.0,5400.10,24.0,5.6776,0.0,0.0,1.0000,0.0000,...,0.00,None,0.000000,5375.300049,5996.3101,2024-11-08 16:41:29,241115,C,600,8.993850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25719,SPXW250930P07400000,1147.2,1.0,1180.20,1.0,0.1160,0.0,0.0,-0.9462,0.0002,...,0.00,None,0.000000,1180.649963,5996.3101,2024-11-08 16:41:29,250930,P,7400,-0.189688
25720,SPXW250930C07600000,8.6,203.0,8.90,14.0,0.1155,71.0,0.0,0.0367,0.0001,...,8.00,2024-11-06T15:45:09,0.000000,8.500000,5996.3101,2024-11-08 16:41:29,250930,C,7600,-0.211012
25721,SPXW250930P07600000,1332.8,1.0,1365.70,1.0,0.1177,1.0,0.0,-0.9680,0.0001,...,1522.36,2024-10-11T10:17:15,0.000000,1367.000000,5996.3101,2024-11-08 16:41:29,250930,P,7600,-0.211012
25722,SPXW250930C07800000,5.4,118.0,5.70,14.0,0.1185,103.0,3.0,0.0238,0.0001,...,5.40,2024-11-08T09:59:25,0.000003,5.400000,5996.3101,2024-11-08 16:41:29,250930,C,7800,-0.231242


# Construct Options Chain
Show the options chain for options near to ATM

In [5]:
# Get unique expiries
expiries = sorted(df['Expiry'].unique())

# Get columns to show in chain
cols = ['bid', 'ask', 'last_trade_price', 'volume', 'iv', 'delta', 'gamma', 'open_interest']

# Print options chain
for exp in expiries:
    # Get call and put data for the expiry, only fairly close to ATM
    expiry_df = df[(df['Expiry'] == exp) & (df['Moneyness'] >= -0.002) & (df['Moneyness'] <= 0.002)]
    call_df = expiry_df[expiry_df['Type'] == 'C'][cols]
    put_df = expiry_df[expiry_df['Type'] == 'P'][cols]
    print(f"{exp} Calls")
    display(call_df)
    print(f"{exp} Puts")
    display(put_df)

241108 Calls


,bid,ask,last_trade_price,volume,iv,delta,gamma,open_interest
8458,14.0,14.3,14.01,14668.0,0.1341,0.7238,0.0180,1016.0
8460,10.5,10.6,10.50,20336.0,0.1281,0.6325,0.0211,5110.0
8462,7.3,7.4,7.40,16754.0,0.1252,0.5238,0.0230,1045.0
8464,4.9,5.0,4.90,34946.0,0.1228,0.4084,0.0227,20672.0
8466,3.1,3.2,3.20,22525.0,0.1228,0.3004,0.0203,12062.0


241108 Puts


,bid,ask,last_trade_price,volume,iv,delta,gamma,open_interest
8459,3.0,3.1,3.04,13335.0,0.1332,-0.2762,0.0180,153.0
8461,4.3,4.4,4.37,11126.0,0.1292,-0.3675,0.0211,173.0
8463,6.2,6.3,6.30,5806.0,0.1254,-0.4762,0.0230,40.0
8465,8.7,8.9,8.84,5374.0,0.1230,-0.5916,0.0227,5909.0
8467,11.9,12.1,11.90,1030.0,0.1220,-0.6996,0.0203,36.0


241111 Calls


...

,bid,ask,last_trade_price,volume,iv,delta,gamma,open_interest
7851,480.3,496.8,490.6,0.0,0.1876,-0.2929,0.0002,879.0


291221 Calls


,bid,ask,last_trade_price,volume,iv,delta,gamma,open_interest
7984,1429.2,1509.0,1452.99,0.0,0.1948,0.7589,0.0001,1517.0


291221 Puts


,bid,ask,last_trade_price,volume,iv,delta,gamma,open_interest
7985,519.9,534.5,533.0,2.0,0.1945,-0.275,0.0001,1397.0


# Gamma Exposure (GEX)
We want to know, at each index level, how much gamma are the market makers sitting on. To do this, we need to know how many of each particular option the market makers hold AND whether they're long or short each option. We can know the first thing from open interest but for the second thingm the data doesn't tell us whether they're long or short (they keep this secret of course!).

So a naive assumption to start with is that the dealers are long calls and short puts, which on the whole index isn't an entirely terrible assumption. We could use actual trades data to construct a more accurate picture of whether they're long or short (based on where trades are being closed relative to the bid, mid, ask can tell you whether the trade is speculative or a hedge for example.). However, this comes at a big cost and isn't easy so for now we'll just assume that the dealers are long calls and short puts.

Recall that long options have positive gamma and short options have negative gamma. So our assumption of long calls and short puts mean that in thie case, calls carry positive gamma and puts negative.

The first thing to do is calculate the gamma contribution of each option. This gives the option's CHANGE IN DELTA per ONE POINT MOVE in the index. The formula is (negate for puts):

    Option Gamma * Contract Size * Open Interest * Index Price
    
However, we actually want how much delta changes per 1% MOVE IN THE INDEX, not 1 point. Since a 1% move corresponds to a point change of 0.01 * Index Price, we can simply multiply our first formula by this giving:

    Option Gamma * Contract Size * Open Interest * (Index Price)^2 * 0.01

Finally, we sum gamma contributions across all of the options to give us the total Gamma Exposure (GEX) of the dealers.

* If GEX is positive, dealers are LONG GAMMA, so if the index moves up 1%, then delta also goes up so dealers need to SELL the underlying to delta hedge. If the index moves down 1%, then delta also goes down so dealers need to BUY the underlying to delta hedge.
* If GEX is negative, dealers are SHORT GAMMA, so if the index moves up 1%, then delta goes down so dealers need to BUY the underlying to delta hedge. If the index moves down 1%, then delta goes up so dealers need to SELL the underlying to delta hedge.

In [6]:
# Calculate gamma contribution of each option TODO? Where is contract size?
sign = df['Type'].apply(lambda x: 1 if x == 'C' else -1)
df['gamma_contribution'] = df['gamma'] * df['open_interest'] * df['Index Price'] * sign

# Calculate GEX
gex = df['gamma_contribution'].sum()
gex

16563883.219494598